In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.4/881.4 kB 23.2 MB/s eta 0:00:00


### Player Tracker (player_tracker.py)

In [ ]:
from ultralytics import YOLO
import cv2
import pickle
class PlayerTracker:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
    def choose_and_filter_players(self, court_keypoints, player_detections):
        player_detections_first_frame = player_detections[0]
        chosen_player = self.choose_players(court_keypoints, player_detections_first_frame)
        filtered_player_detections = []
        for player_dict in player_detections:
            filtered_player_dict = {track_id: bbox for track_id, bbox in player_dict.items() if track_id in chosen_player}
            filtered_player_detections.append(filtered_player_dict)
        return filtered_player_detections
    def choose_players(self, court_keypoints, player_dict):
        distances = []
        for track_id, bbox in player_dict.items():
            player_center = get_center_of_bbox(bbox)

            min_distance = float('inf')
            for i in range(0,len(court_keypoints),2):
                court_keypoint = (court_keypoints[i], court_keypoints[i+1])
                distance = measure_distance(player_center, court_keypoint)
                if distance < min_distance:
                    min_distance = distance
            distances.append((track_id, min_distance))

        # sorrt the distances in ascending order
        distances.sort(key = lambda x: x[1])
        # Choose the first 2 tracks
        chosen_players = [distances[0][0], distances[1][0]]
        return chosen_players


    def detect_frames(self, frames, read_from_stub=False, stub_path=None):
        player_detections = []
        if read_from_stub and stub_path is not None:
            with open(stub_path,'rb') as f:
                player_detections=pickle.load(f)
            return player_detections

        for frame in frames:
            player_dict = self.detect_frame(frame)
            player_detections.append(player_dict)

        if stub_path is not None:
            with open(stub_path,'wb') as f:
                pickle.dump(player_detections,f)

        return player_detections


    def detect_frame(self, frame):
        results = self.model.track(frame, persist=True)[0]
        id_name_dict = results.names
        player_dict = {}
        for box in results.boxes:
            track_id = int(box.id.tolist()[0])
            result = box.xyxy.tolist()[0]
            object_cls_id = box.cls.tolist()[0]
            object_cls_name = id_name_dict[object_cls_id]
            if object_cls_name == "person":
                player_dict[track_id] = result
        return player_dict

    def draw_bboxes(self, video_frames, player_detections):
        output_video_frames = []
        for frame, player_dict in zip(video_frames, player_detections):
            # Draw bounding boxes
            for track_id, bbox in player_dict.items():
                x1, y1, x2, y2 = bbox
                cv2.putText(frame, f"Player ID: {track_id}", (int(bbox[0]), int(bbox[1] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            output_video_frames.append(frame)
        return output_video_frames

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


### Ball Tracker (ball_tracker.py)

In [ ]:
import pandas as pd

class BallTracker:
    def __init__(self, model_path):
        self.model = YOLO(model_path)

    def interpolate_ball_positions(self, ball_positions):
        ball_positions = [x.get(1,[]) for x in ball_positions]
        # convert the list into pandas dataframe
        df_ball_positions = pd.DataFrame(ball_positions,columns=['x1','y1','x2','y2'])

        # interpolate the missing values
        df_ball_positions = df_ball_positions.interpolate()
        df_ball_positions = df_ball_positions.bfill()

        ball_positions = [{1:x} for x in df_ball_positions.to_numpy().tolist()]

        return ball_positions

    def detect_frames(self, frames, read_from_stub=False, stub_path=None):
        ball_detections = []
        if read_from_stub and stub_path is not None:
            with open(stub_path,'rb') as f:
                ball_detections=pickle.load(f)
            return ball_detections

        for frame in frames:
            player_dict = self.detect_frame(frame)
            ball_detections.append(player_dict)

        if stub_path is not None:
            with open(stub_path,'wb') as f:
                pickle.dump(ball_detections,f)

        return ball_detections

    def detect_frame(self, frame):
        results = self.model.predict(frame, conf=0.15)[0]
        ball_dict = {}
        for box in results.boxes:
            result = box.xyxy.tolist()[0]
            ball_dict[1] = result
        return ball_dict

    def draw_bboxes(self, video_frames, ball_detections):
        output_video_frames = []
        for frame, ball_dict in zip(video_frames, ball_detections):
            # Draw bounding boxes
            for track_id, bbox in ball_dict.items():
                x1, y1, x2, y2 = bbox
                cv2.putText(frame, f"Ball ID: {track_id}", (int(bbox[0]), int(bbox[1] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 255), 2)
            output_video_frames.append(frame)
        return output_video_frames

### video_utils.py

In [ ]:
#bbox_utils.py

def get_center_of_bbox(bbox):
    x1, y1, x2, y2 = bbox
    center_x = int((x1 + x2) / 2)
    center_y = int((y1 + y2) / 2)
    return (center_x, center_y)
def measure_distance(p1,p2):
    return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5
def get_foot_position(bbox):
    x1, y1, x2, y2 = bbox
    return (int((x1 + x2) / 2), y2)

def get_closest_keypoint_index(point, keypoints, keypoint_indices):
   closest_distance = float('inf')
   key_point_ind = keypoint_indices[0]
   for keypoint_indix in keypoint_indices:
       keypoint = keypoints[keypoint_indix*2], keypoints[keypoint_indix*2+1]
       distance = abs(point[1]-keypoint[1])

       if distance<closest_distance:
           closest_distance = distance
           key_point_ind = keypoint_indix

   return key_point_ind

def get_height_of_bbox(bbox):
    return bbox[3]-bbox[1]

def measure_xy_distance(p1,p2):
    return abs(p1[0]-p2[0]), abs(p1[1]-p2[1])

def get_center_of_bbox(bbox):
    return (int((bbox[0]+bbox[2])/2),int((bbox[1]+bbox[3])/2))

### Conversions.py


In [ ]:

def convert_pixel_distance_to_meters(pixel_distance, refrence_height_in_meters, refrence_height_in_pixels):
    return (pixel_distance * refrence_height_in_meters) / refrence_height_in_pixels

def convert_meters_to_pixel_distance(meters, refrence_height_in_meters, refrence_height_in_pixels):
    return (meters * refrence_height_in_pixels) / refrence_height_in_meters

In [ ]:
import cv2

def read_video(video_path):
  cap=cv2.VideoCapture(video_path)
  frames=[]
  while cap.isOpened():
    ret,frame=cap.read()
    if not ret:
      break
    frames.append(frame)
  cap.release()
  return frames

def save_video(output_video_frames,output_video_path):
  height, width = output_video_frames[0].shape[:2]
  fourcc=cv2.VideoWriter_fourcc(*'MJPG')
  out=cv2.VideoWriter(output_video_path,fourcc,24,(width,height))
  for frame in output_video_frames:
    out.write(frame)
  out.release()

In [ ]:
!unzip tennis_court_det_dataset.zip

unzip:  cannot find or open tennis_court_det_dataset.zip, tennis_court_det_dataset.zip.zip or tennis_court_det_dataset.zip.ZIP.


In [ ]:
import torch
from torch.utils.data import  Dataset, DataLoader
from torchvision  import transforms, utils, models

import json
import cv2
import numpy as np

dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(dev)

cuda


In [ ]:
class KeypointCNN(nn.Module):
    def __init__(self):
        super(KeypointCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 14 * 2)  # 14 keypoints (x, y)

    def forward(self, x):
        # Forward pass
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        
        # Flattening output
        x = x.view(-1, 128 * 28 * 28)
        
        # Forward pass through fully connected layers
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, 'r') as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2]
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        # Adjust coordinates to the new image size
        kps[::2] = 224.0 / w * kps[::2]
        kps[1::2] = 224.0 / h * kps[1::2]

        return img, kps

In [ ]:
train_dataset = KeypointsDataset("data/images",'data/data_train.json')
val_dataset = KeypointsDataset("data/images",'data/data_val.json')

train_loader = DataLoader(train_dataset ,batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

FileNotFoundError: [Errno 2] No such file or directory: 'data/data_train.json'

In [ ]:
# Create model, loss function, and optimizer
model = KeypointCNN().to(dev)  # Replace ResNet with custom CNN
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    for i, (imgs, kps) in enumerate(train_loader):
        imgs = imgs.to(dev)
        kps = kps.to(dev)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

# Saving the trained model
torch.save(model.state_dict(), 'keypoints_model.pth')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 124MB/s]


NameError: name 'train_loader' is not defined

### Tennis Court Keypoints Training

### Court Line Detector (court_line_detector.py)

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt

class CourtLineDetector:
    def __init__(self, model_path):
        self.model = models.resnet50(pretrained=True)
        self.model.fc = torch.nn.Linear(self.model.fc.in_features, 14*2)
        self.model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')), strict=False)

        self.transforms = transforms.Compose([
          transforms.ToPILImage(),
          transforms.Resize((224, 224)),
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def predict(self, image):
        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_tensor = self.transforms(img_rgb)
        image_tensor = image_tensor.unsqueeze(0)

        with torch.no_grad():
          outputs = self.model(image_tensor)

        keypoints = outputs.squeeze().numpy()
        org_h, org_w = img_rgb.shape[:2]

        keypoints[::2] *= org_w/224.0
        keypoints[1::2] *= org_h/224.0

        return keypoints

    def draw_keypoints(self, image, keypoints):
        for i in range(0, len(keypoints), 2):
          x, y = int(keypoints[i]), int(keypoints[i+1])

          cv2.putText(image, str(i//2), (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
          cv2.circle(image, (x, y), 5, (0, 0, 255), -1)
        return image

    def draw_keypoints_on_video(self, video_frames, keypoints):
        output_video_frames = []
        for frame in video_frames:
          frame = self.draw_keypoints(frame, keypoints)
          output_video_frames.append(frame)
        return output_video_frames

### Defining constants for drawing the mini court



In [ ]:
SINGLE_LINE_WIDTH = 8.23
DOUBLE_LINE_WIDTH = 10.97
HALF_COURT_LINE_HEIGHT = 11.88
SERVICE_LINE_WIDTH = 6.4
DOUBLE_ALLY_DIFFERENCE = 1.37
NO_MANS_LAND_HEIGHT = 5.48

PLAYER_1_HEIGHT_METERS = 1.88
PLAYER_2_HEIGHT_METERS = 1.91

### mini_court.py


In [ ]:
import cv2
class MiniCourt():
  def __init__(self,frame):
          self.drawing_rectangle_width = 250
          self.drawing_rectangle_height = 500
          self.buffer = 50
          self.padding_court=20

          self.set_canvas_background_box_position(frame)
          self.set_mini_court_position()
          self.set_court_drawing_key_points()
          self.set_court_lines()
  def convert_meters_to_pixels(self, meters):
          return convert_meters_to_pixel_distance(meters,
                                                  DOUBLE_LINE_WIDTH,
                                                  self.court_drawing_width
                                              )
  def set_court_drawing_key_points(self):
          drawing_key_points = [0]*28
          # point 0
          drawing_key_points[0] , drawing_key_points[1] = int(self.court_start_x), int(self.court_start_y)
          # point 1
          drawing_key_points[2] , drawing_key_points[3] = int(self.court_end_x), int(self.court_start_y)
          # point 2
          drawing_key_points[4] = int(self.court_start_x)
          drawing_key_points[5] = self.court_start_y + self.convert_meters_to_pixels(HALF_COURT_LINE_HEIGHT*2)
          # point 3
          drawing_key_points[6] = drawing_key_points[0] + self.court_drawing_width
          drawing_key_points[7] = drawing_key_points[5]
          # #point 4
          drawing_key_points[8] = drawing_key_points[0] +  self.convert_meters_to_pixels(DOUBLE_ALLY_DIFFERENCE)
          drawing_key_points[9] = drawing_key_points[1]
          # #point 5
          drawing_key_points[10] = drawing_key_points[4] + self.convert_meters_to_pixels(DOUBLE_ALLY_DIFFERENCE)
          drawing_key_points[11] = drawing_key_points[5]
          # #point 6
          drawing_key_points[12] = drawing_key_points[2] - self.convert_meters_to_pixels(DOUBLE_ALLY_DIFFERENCE)
          drawing_key_points[13] = drawing_key_points[3]
          # #point 7
          drawing_key_points[14] = drawing_key_points[6] - self.convert_meters_to_pixels(DOUBLE_ALLY_DIFFERENCE)
          drawing_key_points[15] = drawing_key_points[7]
          # #point 8
          drawing_key_points[16] = drawing_key_points[8]
          drawing_key_points[17] = drawing_key_points[9] + self.convert_meters_to_pixels(NO_MANS_LAND_HEIGHT)
          # # #point 9
          drawing_key_points[18] = drawing_key_points[16] + self.convert_meters_to_pixels(SINGLE_LINE_WIDTH)
          drawing_key_points[19] = drawing_key_points[17]
          # #point 10
          drawing_key_points[20] = drawing_key_points[10]
          drawing_key_points[21] = drawing_key_points[11] - self.convert_meters_to_pixels(NO_MANS_LAND_HEIGHT)
          # # #point 11
          drawing_key_points[22] = drawing_key_points[20] +  self.convert_meters_to_pixels(SINGLE_LINE_WIDTH)
          drawing_key_points[23] = drawing_key_points[21]
          # # #point 12
          drawing_key_points[24] = int((drawing_key_points[16] + drawing_key_points[18])/2)
          drawing_key_points[25] = drawing_key_points[17]
          # # #point 13
          drawing_key_points[26] = int((drawing_key_points[20] + drawing_key_points[22])/2)
          drawing_key_points[27] = drawing_key_points[21]

          self.drawing_key_points=drawing_key_points

  def convert_bounding_boxes_to_mini_court_coordinates(self,player_boxes, ball_boxes, original_court_key_points ):
        player_heights = {
            1: PLAYER_1_HEIGHT_METERS,
            2: PLAYER_2_HEIGHT_METERS
        }

        output_player_boxes= []
        output_ball_boxes= []

        for frame_num, player_bbox in enumerate(player_boxes):
            ball_box = ball_boxes[frame_num][1]
            ball_position = get_center_of_bbox(ball_box)

            #Handle cases with more than two players by selecting the two closest to the ball.
            closest_player_ids_to_ball = sorted(player_bbox.keys(), key=lambda x: measure_distance(ball_position, get_center_of_bbox(player_bbox[x])))[:2]

            output_player_bboxes_dict = {}
            for player_id in closest_player_ids_to_ball: # Only iterate over the two closest players
                bbox = player_bbox[player_id]
                foot_position = get_foot_position(bbox)

                # Get The closest keypoint in pixels
                closest_key_point_index = get_closest_keypoint_index(foot_position,original_court_key_points, [0,2,12,13])
                closest_key_point = (original_court_key_points[closest_key_point_index*2],
                                     original_court_key_points[closest_key_point_index*2+1])

                # Get Player height in pixels
                frame_index_min = max(0, frame_num-20)
                frame_index_max = min(len(player_boxes), frame_num+50)
                bboxes_heights_in_pixels = [get_height_of_bbox(player_boxes[i][player_id]) for i in range (frame_index_min,frame_index_max)]
                max_player_height_in_pixels = max(bboxes_heights_in_pixels)

                mini_court_player_position = self.get_mini_court_coordinates(foot_position,
                                                                            closest_key_point,
                                                                            closest_key_point_index,
                                                                            max_player_height_in_pixels,
                                                                            player_heights.get(player_id, 1.88) # Use a default height if player_id not in dict
                                                                            )

                output_player_bboxes_dict[player_id] = mini_court_player_position

                if player_id in closest_player_ids_to_ball:
                    # Get The closest keypoint in pixels
                    closest_key_point_index = get_closest_keypoint_index(ball_position,original_court_key_points, [0,2,12,13])
                    closest_key_point = (original_court_key_points[closest_key_point_index*2],
                                        original_court_key_points[closest_key_point_index*2+1])

                    mini_court_player_position = self.get_mini_court_coordinates(ball_position,
                                                                            closest_key_point,
                                                                            closest_key_point_index,
                                                                            max_player_height_in_pixels,
                                                                            player_heights.get(player_id, 1.88) # Use a default height if player_id not in dict
                                                                            )
                    output_ball_boxes.append({1:mini_court_player_position})
            output_player_boxes.append(output_player_bboxes_dict)

        return output_player_boxes , output_ball_boxes

        return output_player_boxes , output_ball_boxes
  def get_mini_court_coordinates(self,
                                   object_position,
                                   closest_key_point,
                                   closest_key_point_index,
                                   player_height_in_pixels,
                                   player_height_in_meters
                                   ):

        distance_from_keypoint_x_pixels, distance_from_keypoint_y_pixels = measure_xy_distance(object_position, closest_key_point)

        # Conver pixel distance to meters
        distance_from_keypoint_x_meters = convert_pixel_distance_to_meters(distance_from_keypoint_x_pixels,
                                                                           player_height_in_meters,
                                                                           player_height_in_pixels
                                                                           )
        distance_from_keypoint_y_meters = convert_pixel_distance_to_meters(distance_from_keypoint_y_pixels,
                                                                                player_height_in_meters,
                                                                                player_height_in_pixels
                                                                          )

        # Convert to mini court coordinates
        mini_court_x_distance_pixels = self.convert_meters_to_pixels(distance_from_keypoint_x_meters)
        mini_court_y_distance_pixels = self.convert_meters_to_pixels(distance_from_keypoint_y_meters)
        closest_mini_coourt_keypoint = ( self.drawing_key_points[closest_key_point_index*2],
                                        self.drawing_key_points[closest_key_point_index*2+1]
                                        )

        mini_court_player_position = (closest_mini_coourt_keypoint[0]+mini_court_x_distance_pixels,
                                      closest_mini_coourt_keypoint[1]+mini_court_y_distance_pixels
                                        )

        return  mini_court_player_position


  def set_court_lines(self):
        self.lines = [
            (0, 2),
            (4, 5),
            (6,7),
            (1,3),

            (0,1),
            (8,9),
            (10,11),
            (10,11),
            (2,3)
        ]
  def set_mini_court_position(self):
          self.court_start_x = self.start_x + self.padding_court
          self.court_start_y = self.start_y + self.padding_court
          self.court_end_x = self.end_x - self.padding_court
          self.court_end_y = self.end_y - self.padding_court
          self.court_drawing_width = self.court_end_x - self.court_start_x


  def set_canvas_background_box_position(self,frame):
          frame= frame.copy()

          self.end_x = frame.shape[1] - self.buffer
          self.end_y = self.buffer + self.drawing_rectangle_height
          self.start_x = self.end_x - self.drawing_rectangle_width
          self.start_y = self.end_y - self.drawing_rectangle_height
  def draw_court(self,frame):
        for i in range(0, len(self.drawing_key_points),2):
            x = int(self.drawing_key_points[i])
            y = int(self.drawing_key_points[i+1])
            cv2.circle(frame, (x,y),5, (0,0,255),-1)

        # draw Lines
        for line in self.lines:
            start_point = (int(self.drawing_key_points[line[0]*2]), int(self.drawing_key_points[line[0]*2+1]))
            end_point = (int(self.drawing_key_points[line[1]*2]), int(self.drawing_key_points[line[1]*2+1]))
            cv2.line(frame, start_point, end_point, (0, 0, 0), 2)

        # Draw net
        net_start_point = (self.drawing_key_points[0], int((self.drawing_key_points[1] + self.drawing_key_points[5])/2))
        net_end_point = (self.drawing_key_points[2], int((self.drawing_key_points[1] + self.drawing_key_points[5])/2))
        cv2.line(frame, net_start_point, net_end_point, (255, 0, 0), 2)

        return frame
  def draw_background_rectangle(self,frame):
        shapes = np.zeros_like(frame,np.uint8)
        # Draw the rectangle
        cv2.rectangle(shapes, (self.start_x, self.start_y), (self.end_x, self.end_y), (255, 255, 255), cv2.FILLED)
        out = frame.copy()
        alpha=0.5
        mask = shapes.astype(bool)
        out[mask] = cv2.addWeighted(frame, alpha, shapes, 1 - alpha, 0)[mask]

        return out
  def draw_mini_court(self,frames):
        output_frames = []
        for frame in frames:
            frame = self.draw_background_rectangle(frame)
            frame = self.draw_court(frame)
            output_frames.append(frame)
        return output_frames
  def draw_points_on_mini_court(self,frames,postions, color=(0,255,0)):
        for frame_num, frame in enumerate(frames):
            for _, position in postions[frame_num].items():
                x,y = position
                x= int(x)
                y= int(y)
                cv2.circle(frame, (x,y), 5, color, -1)
        return frames


### main.py

In [ ]:

input_video_path="/content/drive/MyDrive/AI_Project_Models/input_video.mp4"
video_frames = read_video(input_video_path)

player_tracker = PlayerTracker(model_path='yolov8x')
ball_tracker = BallTracker(model_path='/content/drive/MyDrive/AI_Project_Models/yolov5_best.pt')

player_detections=player_tracker.detect_frames(video_frames,
                                               read_from_stub=True,
                                               stub_path='/content/drive/MyDrive/AI_Project_Models/tracker_stubs/player_detections.pkl')
ball_detections=ball_tracker.detect_frames(video_frames,
                                           read_from_stub=True,
                                           stub_path='/content/drive/MyDrive/AI_Project_Models/tracker_stubs/ball_detections.pkl')
ball_detections = ball_tracker.interpolate_ball_positions(ball_detections)

court_line_detector = CourtLineDetector(model_path='/content/drive/MyDrive/AI_Project_Models/keypoints_model.pth')
court_keypoints = court_line_detector.predict(video_frames[0])
import cv2
output_video_frames = player_tracker.draw_bboxes(video_frames,player_detections)
output_video_frames = ball_tracker.draw_bboxes(output_video_frames,ball_detections)
output_video_frames = court_line_detector.draw_keypoints_on_video(video_frames,court_keypoints)

# Draw Mini Court
mini_court = MiniCourt(video_frames[0])
player_mini_court_detections,ball_mini_court_detections = mini_court.convert_bounding_boxes_to_mini_court_coordinates(player_detections,
                                                                                                          ball_detections,
                                                                                                          court_keypoints)

output_video_frames = mini_court.draw_mini_court(output_video_frames)
output_video_frames = mini_court.draw_points_on_mini_court(output_video_frames,player_mini_court_detections)
output_video_frames = mini_court.draw_points_on_mini_court(output_video_frames,ball_mini_court_detections, color=(0,255,255))

#frame number top left corner
for i, frame in enumerate(output_video_frames):
      cv2.putText(frame, f"Frame: {i}",(10,30),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
save_video(output_video_frames,"/content/drive/MyDrive/AI_Project_Models/output_video_interpolation.avi")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
